In [1]:
credentials_file = ".encrypted_credentials"

In [2]:
key = input("Inserire la chiave di cifratura: ")

In [3]:
out_folder = "dataset"

In [4]:
from cryptography.fernet import Fernet
import json

def decrypt_credentials(encrypted_credentials_path, key):
    with open(encrypted_credentials_path, 'rb') as file:
        encrypted_credentials = file.read()

    cipher_suite = Fernet(key)
    decrypted_credentials = cipher_suite.decrypt(encrypted_credentials)

    return decrypted_credentials

decrypted_credentials = decrypt_credentials(credentials_file, key.encode())
credentials_dict = json.loads(decrypted_credentials.decode('utf-8'))

In [8]:
import os
from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload
import io


credentials = service_account.Credentials.from_service_account_info(
    credentials_dict,
    scopes=['https://www.googleapis.com/auth/drive'],
)

drive_service = build('drive', 'v3', credentials=credentials)

folder_id = '1MB_jsX3ydVCDXmf-48leE0BTigY4tMGW'

def download_files(drive_service, folder_id, output_dir, c):
    page_token = None
    while True:
        response = drive_service.files().list(
            q=f"'{folder_id}' in parents",
            spaces='drive',
            fields='nextPageToken, files(id, name, mimeType)',
            pageToken=page_token
        ).execute()


        for file in response.get('files', []):
            file_id = file['id']
            file_name = file['name']
            mime_type = file['mimeType']

            if mime_type == 'application/vnd.google-apps.folder':
                download_files(drive_service, file_id, os.path.join(output_dir, file_name), c)
            else:
                request = drive_service.files().get_media(fileId=file_id)
                file_path = os.path.join(output_dir, file_name)
                if not os.path.exists(file_path):
                    fh = io.FileIO(file_path, 'wb')
                    downloader = MediaIoBaseDownload(fh, request, chunksize=10*1024*1024)
                    downloader.timeout = 2000
                    done = False
                    while not done:
                        try:
                            status, done = downloader.next_chunk()
                            print(f"Download {file_name}: {int(status.progress() * 100)}% [{c}]")
                        except:
                            print(f"\t\t\t->Errore durante il download di {file_name} [{c}]")
                            fh.close()
                    fh.close()
                c+=1
                

        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

c = 1
download_files(drive_service, folder_id, out_folder, c)
print("Download dell'intera cartella completato!")

Download dell'intera cartella completato!
